#### Author: Sailesh Chauhan
#### Date: 25-07-2021
#### Title: CPU_logs dataset has been analysed to find out solution for following problem statement.

1. Finding users with lowest number of average hours using Data Bricks sql
2. Finding users with highest number of average hours using Data Bricks sql
3. Finding users with highest numbers of times late comings using Data Bricks sql
4. Finding users with highest numbers of idle hours using Data Bricks sql
5. Store the cleaning data on BLOB Storage
6. Store results on BLOB Storage

In [0]:
%fs
ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/CpuLogData2019_09_16.csv,CpuLogData2019_09_16.csv,177697
dbfs:/FileStore/tables/CpuLogData2019_09_17.csv,CpuLogData2019_09_17.csv,299062
dbfs:/FileStore/tables/CpuLogData2019_09_18.csv,CpuLogData2019_09_18.csv,286264
dbfs:/FileStore/tables/CpuLogData2019_09_19.csv,CpuLogData2019_09_19.csv,346015
dbfs:/FileStore/tables/CpuLogData2019_09_20.csv,CpuLogData2019_09_20.csv,136532
dbfs:/FileStore/tables/CpuLogData2019_09_21.csv,CpuLogData2019_09_21.csv,291933
dbfs:/FileStore/tables/LMS_DB.zip,LMS_DB.zip,636912


In [0]:
# Mounting Azure Blob Storage
AZURE_KEY=spark.conf.get('spark.azure_key')
dbutils.fs.mount(
  source = "wasbs://cpulogsdata@cpulogs.blob.core.windows.net",
  mount_point = "/mnt/CPU_Logs_New",
  extra_configs ={"fs.azure.account.key.cpulogs.blob.core.windows.net":AZURE_KEY})

Out[4]: True

In [0]:
%fs
ls /mnt

path,name,size
dbfs:/mnt/CPU_Logs/,CPU_Logs/,0
dbfs:/mnt/CPU_Logs_New/,CPU_Logs_New/,0


In [0]:
# File location as list of file paths loading all files into data frame

file_location = ["/FileStore/tables/CpuLogData2019_09_16.csv","/FileStore/tables/CpuLogData2019_09_17.csv","/FileStore/tables/CpuLogData2019_09_18.csv","/FileStore/tables/CpuLogData2019_09_19.csv","/FileStore/tables/CpuLogData2019_09_20.csv","/FileStore/tables/CpuLogData2019_09_21.csv"]

file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Selecting only required columns from dataset.

df_CPU_logs=df.select("DateTime","user_name","keyboard","mouse")
rows=df_CPU_logs.count()
df_CPU_logs.describe()
rows

Out[5]: 4122

In [0]:
# Adding column time containing values in YYYY-MM-DD HH:mm from TimeStamp
from pyspark.sql.functions import date_format
df_CPU_logs_date_time = df_CPU_logs.withColumn('time', date_format('datetime', 'yyyy-MM-dd HH:mm'))


# Dropping date time columns
df_CPU_logs_date_time=df_CPU_logs_date_time.drop("DateTime")
df_CPU_logs_date_time.show()

+--------------------+--------+------+----------------+
 user_name|keyboard| mouse| time|
+--------------------+--------+------+----------------+
 iamnzm@outlook.com| 1.0| 32.0|2019-09-19 08:40|
 iamnzm@outlook.com| 0.0| 0.0|2019-09-19 08:45|
 iamnzm@outlook.com| 0.0| 0.0|2019-09-19 08:50|
 iamnzm@outlook.com| 11.0| 900.0|2019-09-19 08:55|
 iamnzm@outlook.com| 2.0| 25.0|2019-09-19 09:00|
 iamnzm@outlook.com| 37.0| 336.0|2019-09-19 09:05|
deepshukla292@gma...| 0.0| 55.0|2019-09-19 09:05|
 iamnzm@outlook.com| 0.0| 136.0|2019-09-19 09:10|
deepshukla292@gma...| 6.0|1112.0|2019-09-19 09:10|
 iamnzm@outlook.com| 0.0| 84.0|2019-09-19 09:15|
deepshukla292@gma...| 0.0| 357.0|2019-09-19 09:15|
markfernandes66@g...| 20.0| 670.0|2019-09-19 09:15|
markfernandes66@g...| 17.0| 0.0|2019-09-19 09:10|
markfernandes66@g...| 29.0|1895.0|2019-09-19 09:20|
 iamnzm@outlook.com| 0.0| 29.0|2019-09-19 09:20|
deepshukla292@gma...| 35.0|3022.0|2019-09-19 09:20|
markfernandes66@g...| 2.0| 238.0|2019-09-19 09:25|
 iamnzm@outlook.com| 0.0| 0.0|2019-09-19 09:25|
deepshukla292@gma...| 20.0|1330.0|2019-09-19 09:25|
markfernandes66@g...| 5.0| 39.0|2019-09-19 09:30|
+--------------------+--------+------+----------------+
only showing top 20 rows

In [0]:
# Creating a parmanent table CpuLogData_All1

permanent_table_name = "CpuLogData_All1"

df_CPU_logs_date_time.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
%sql
--sql query to count total rows
SELECT COUNT(*) FROM CpuLogData_All1

count(1)
4122


In [0]:
%sql
-- Analyze table and compute statistics.
ANALYZE TABLE CpuLogData_All1 COMPUTE STATISTICS;
DESC EXTENDED CpuLogData_All1;

col_name,data_type,comment
user_name,string,null
keyboard,string,null
mouse,string,null
time,string,null
,,
# Detailed Table Information,,
Database,default,
Table,cpulogdata_all1,
Owner,root,
Created Time,Sun Jul 25 11:28:50 UTC 2021,


In [0]:
%sql
-- 10 reocrds of table
SELECT  * FROM CpuLogData_All1 limit 10;

user_name,keyboard,mouse,time
iamnzm@outlook.com,1.0,32.0,2019-09-19 08:40
iamnzm@outlook.com,0.0,0.0,2019-09-19 08:45
iamnzm@outlook.com,0.0,0.0,2019-09-19 08:50
iamnzm@outlook.com,11.0,900.0,2019-09-19 08:55
iamnzm@outlook.com,2.0,25.0,2019-09-19 09:00
iamnzm@outlook.com,37.0,336.0,2019-09-19 09:05
deepshukla292@gmail.com,0.0,55.0,2019-09-19 09:05
iamnzm@outlook.com,0.0,136.0,2019-09-19 09:10
deepshukla292@gmail.com,6.0,1112.0,2019-09-19 09:10
iamnzm@outlook.com,0.0,84.0,2019-09-19 09:15


In [0]:
# Writing data to mounted Azure Blob Storage

df=spark.sql("""SELECT  * FROM CpuLogData_All1""")
df.write.csv('dbfs:/mnt/CPU_Logs/CpuLogData_All1')

+--------------------+--------+------+----------------+
 user_name|keyboard| mouse| time|
+--------------------+--------+------+----------------+
 iamnzm@outlook.com| 1.0| 32.0|2019-09-19 08:40|
 iamnzm@outlook.com| 0.0| 0.0|2019-09-19 08:45|
 iamnzm@outlook.com| 0.0| 0.0|2019-09-19 08:50|
 iamnzm@outlook.com| 11.0| 900.0|2019-09-19 08:55|
 iamnzm@outlook.com| 2.0| 25.0|2019-09-19 09:00|
 iamnzm@outlook.com| 37.0| 336.0|2019-09-19 09:05|
deepshukla292@gma...| 0.0| 55.0|2019-09-19 09:05|
 iamnzm@outlook.com| 0.0| 136.0|2019-09-19 09:10|
deepshukla292@gma...| 6.0|1112.0|2019-09-19 09:10|
 iamnzm@outlook.com| 0.0| 84.0|2019-09-19 09:15|
deepshukla292@gma...| 0.0| 357.0|2019-09-19 09:15|
markfernandes66@g...| 20.0| 670.0|2019-09-19 09:15|
markfernandes66@g...| 17.0| 0.0|2019-09-19 09:10|
markfernandes66@g...| 29.0|1895.0|2019-09-19 09:20|
 iamnzm@outlook.com| 0.0| 29.0|2019-09-19 09:20|
deepshukla292@gma...| 35.0|3022.0|2019-09-19 09:20|
markfernandes66@g...| 2.0| 238.0|2019-09-19 09:25|
 iamnzm@outlook.com| 0.0| 0.0|2019-09-19 09:25|
deepshukla292@gma...| 20.0|1330.0|2019-09-19 09:25|
markfernandes66@g...| 5.0| 39.0|2019-09-19 09:30|
+--------------------+--------+------+----------------+
only showing top 20 rows

In [0]:
%sql
CREATE table cpu_idle_working1 AS (SELECT `user_name`,bigint(to_timestamp(`time`)) AS datetime_bigint,
case
when (`keyboard`>0 OR `mouse`>0) then 'working'
else 'idle'
end AS Work_Status
FROM `CpuLogData_All1`)

num_affected_rows,num_inserted_rows


In [0]:
df=spark.sql("""SELECT * FROM cpu_idle_working1""")
df.write.csv('dbfs:/mnt/CPU_Logs/cpu_idle_working1')

In [0]:
%sql
SELECT * FROM cpu_idle_working1 limit 10;

user_name,datetime_bigint,Work_Status
iamnzm@outlook.com,1568882400,working
iamnzm@outlook.com,1568882700,idle
iamnzm@outlook.com,1568883000,idle
iamnzm@outlook.com,1568883300,working
iamnzm@outlook.com,1568883600,working
iamnzm@outlook.com,1568883900,working
deepshukla292@gmail.com,1568883900,working
iamnzm@outlook.com,1568884200,working
deepshukla292@gmail.com,1568884200,working
iamnzm@outlook.com,1568884500,working


In [0]:
%sql
SELECT * FROM cpu_idle_working1 where work_status='idle' limit 10

user_name,datetime_bigint,Work_Status
iamnzm@outlook.com,1568882700,idle
iamnzm@outlook.com,1568883000,idle
iamnzm@outlook.com,1568885100,idle
iamnzm@outlook.com,1568885400,idle
markfernandes66@gmail.com,1568885700,idle
markfernandes66@gmail.com,1568886000,idle
markfernandes66@gmail.com,1568886300,idle
iamnzm@outlook.com,1568887200,idle
markfernandes66@gmail.com,1568888100,idle
damodharn21@gmail.com,1568890200,idle


In [0]:
%sql
SELECT * FROM cpu_idle_working1 where work_status='working' limit 10

user_name,datetime_bigint,Work_Status
iamnzm@outlook.com,1568882400,working
iamnzm@outlook.com,1568883300,working
iamnzm@outlook.com,1568883600,working
iamnzm@outlook.com,1568883900,working
deepshukla292@gmail.com,1568883900,working
iamnzm@outlook.com,1568884200,working
deepshukla292@gmail.com,1568884200,working
iamnzm@outlook.com,1568884500,working
deepshukla292@gmail.com,1568884500,working
markfernandes66@gmail.com,1568884500,working


In [0]:
%sql
ANALYZE TABLE cpu_idle_working1 COMPUTE STATISTICS;
DESC EXTENDED cpu_idle_working1;

col_name,data_type,comment
user_name,string,
datetime_bigint,bigint,
Work_Status,string,
,,
# Partitioning,,
Not partitioned,,
,,
# Detailed Table Information,,
Name,default.cpu_idle_working1,
Location,dbfs:/user/hive/warehouse/cpu_idle_working1,


In [0]:
%sql
SELECT COUNT(*) FROM cpu_idle_working1

count(1)
4122


In [0]:
%sql
CREATE table user_idle_time1 AS (SELECT user_name,datetime_bigint,Work_Status,
datetime_bigint - lag(datetime_bigint) over (order by datetime_bigint) AS time_idle_no_work
FROM cpu_idle_working1 where work_status='idle')

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM user_idle_time1 limit 10

user_name,datetime_bigint,Work_Status,time_idle_no_work
rahilstar11@gmail.com,1568638500,idle,null
iamnzm@outlook.com,1568640000,idle,1500
rahilstar11@gmail.com,1568640300,idle,300
iamnzm@outlook.com,1568640300,idle,0
sharlawar77@gmail.com,1568640300,idle,0
rahilstar11@gmail.com,1568640600,idle,300
iamnzm@outlook.com,1568640600,idle,0
deepshukla292@gmail.com,1568640600,idle,0
sharlawar77@gmail.com,1568640600,idle,0
rahilstar11@gmail.com,1568640900,idle,300


In [0]:
%sql
-- User average idle hours with user_name
SELECT from_unixtime(round((sum(time_idle_no_work)/6),2),'HH:mm') AS user_Avg_idle_hours,user_name FROM user_idle_time1 group by user_name order by user_Avg_idle_hours desc

user_Avg_idle_hours,user_name
05:57,sharlawar77@gmail.com
05:45,iamnzm@outlook.com
03:32,deepshukla292@gmail.com
01:47,rahilstar11@gmail.com
01:30,salinabodale73@gmail.com
01:03,bhagyashrichalke21@gmail.com
00:49,damodharn21@gmail.com
00:37,markfernandes66@gmail.com


In [0]:
# Writing above table as csv file to Azure blob storage
df=spark.sql("""SELECT from_unixtime(round(sum(time_idle_no_work)/(6),3),'HH:mm') AS user_Avg_idle_hours,user_name FROM user_idle_time1 group by user_name order by user_Avg_idle_hours desc """)
df.show()
df.write.option.csv("header",true,'dbfs:/mnt/CPU_Logs/user_Avg_idle_hours')

+-------------------+--------------------+
user_Avg_idle_hours| user_name|
+-------------------+--------------------+
 05:57|sharlawar77@gmail...|
 05:45| iamnzm@outlook.com|
 03:32|deepshukla292@gma...|
 01:47|rahilstar11@gmail...|
 01:30|salinabodale73@gm...|
 01:03|bhagyashrichalke2...|
 00:49|damodharn21@gmail...|
 00:37|markfernandes66@g...|
+-------------------+--------------------+

In [0]:
%sql
CREATE table user_working_time1 AS (SELECT user_name,datetime_bigint,Work_Status,
datetime_bigint - lag(datetime_bigint) over (order by datetime_bigint) AS time_user_working
FROM cpu_idle_working1 where work_status='working')

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM user_working_time1

count(1)
3167


In [0]:
%sql
SELECT from_unixtime(round(sum(time_user_working)/6,3),'HH:mm') AS user_Avg_work_hours,user_name FROM user_working_time1 group by user_name order by user_Avg_work_hours desc

user_Avg_work_hours,user_name
06:56,iamnzm@outlook.com
03:49,sharlawar77@gmail.com
03:38,deepshukla292@gmail.com
03:29,salinabodale73@gmail.com
01:41,rahilstar11@gmail.com
00:35,markfernandes66@gmail.com
00:33,damodharn21@gmail.com
00:23,bhagyashrichalke21@gmail.com


In [0]:

df = spark.sql("""
            SELECT from_unixtime(round(sum(time_user_working)/6,3),'HH:mm') AS user_Avg_work_hours,user_name FROM user_working_time1 group by user_name order by user_Avg_work_hours desc
            """)
df.show()
df.write.option.csv("header",true,'dbfs:/mnt/CPU_Logs/user_Avg_idle_hours')

+-------------------+--------------------+
user_Avg_work_hours| user_name|
+-------------------+--------------------+
 06:56| iamnzm@outlook.com|
 03:49|sharlawar77@gmail...|
 03:38|deepshukla292@gma...|
 03:29|salinabodale73@gm...|
 01:41|rahilstar11@gmail...|
 00:35|markfernandes66@g...|
 00:33|damodharn21@gmail...|
 00:23|bhagyashrichalke2...|
+-------------------+--------------------+